In [50]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.preprocessing import scale
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV

import herv_preprocess as hpp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Parameter configuration

In [131]:
# input data
filename = 'data/304_sessions_e.csv'
# k-fold cross validation (one in each k examples will be used for validation, the others for training)
k = 5

dtype = "i4,U20,U5,f4,f4,f4,f4,f4,f4,f4,f4,f4,f4,f4,f4,f4,f4,f4,f4,f4,f4"
dtype = "i4,U20,U5,f4,f4,f4,f4,f4,f4,f4,f4,f4,f4,f4,f4,f4,f4,f4"
dtype = "i4,U20,U5,f4,f4,f4,f4,f4,f4,f4,f4,f4"

features=['AVGNN','SDNN','MeanHR','SDHR','MinHR','MaxHR','RMSSD','NNxx','pNNxx', 'triangularindex', 'TINN', 'powerVLF', 'powerLF', 'powerHF', 'ratioHFLF', 'peakVLF', 'peakLF', 'peakHF']

### 1 - Extract data from the csv file and split it into labels and features

In [132]:
labels, examples = hpp.getData(filename, dtype)
h1 = hpp.addHierarchy(labels, group1=('movement', 'household-chores'))#, name1='moving', name2='still')
print(len(h1))

(#samples, #features) =  (300, 9)
min and max values before scaling:  0.0 1135.40002441
min and max values after scaling:  -2.67828367187 4.97289409218
300


In [133]:
ltrain, ltest, ftrain, ftest = hpp.trainAndTestDatasets(h1, examples, k )

g1: 70 examples (56 for train and 14 for test)
g2: 230 examples (184 for train and 46 for test)

Total: 240 train examples and 60 test examples 


### 2 - test classifiers with no cross validation and predefined parameters (C=1, $\gamma$ = 0.01)

In [134]:
clf1 = svm.SVC(kernel='linear', cache_size=1000, C=10)
clf1.fit(X=ftrain, y=ltrain)
hpp.printResults(ltest, clf1.predict(ftest))

expected		result
------------------------------
g1		g1
g1		g1
g1		g1
g1		g1
g1		g2
g1		g2
g1		g2
g1		g1
g1		g1
g1		g1
g1		g1
g1		g1
g1		g1
g1		g1
g2		g2
g2		g2
g2		g2
g2		g2
g2		g1
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g1
g2		g2
g2		g2
g2		g1
g2		g2
g2		g1
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g1
g2		g1
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
------
Got 51 out of 60 right! :)


In [135]:
clf2 = svm.SVC(kernel='rbf', cache_size=1000, C=1, gamma=0.01)
clf2.fit(X=ftrain, y=ltrain)
hpp.printResults(ltest, clf2.predict(ftest))

expected		result
------------------------------
g1		g1
g1		g1
g1		g2
g1		g1
g1		g2
g1		g2
g1		g2
g1		g1
g1		g1
g1		g1
g1		g1
g1		g1
g1		g1
g1		g1
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g1
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
g2		g2
------
Got 55 out of 60 right! :)


### 3 - Model selection:  Perform an exhaustive search, with k-fold cross-validation,  in the space parameter consisting of:
* $C = 10^{i}, com -1 \leq i \leq 6$, for both linear and RBF kernels
* $\gamma = 10^{i}, com -3 \leq i \leq 3$, for RBF kernel.

In [94]:
crossval = StratifiedShuffleSplit(n_splits=k-1, test_size=0.2)

# C varies from 10^(-1) to 10^6, gamma from 10^(-3) to 10^3
c_range = np.logspace(-1, 2, 4) 
gamma_range = np.logspace(-2, 1, 4)

param_lin=dict(C=c_range)
param_rbf=dict(C=c_range, gamma=gamma_range)


In [99]:
grid_lin = GridSearchCV(svm.SVC(kernel='linear', cache_size=1000), param_grid=param_lin, cv=crossval)
grid_lin.fit(X=ftrain, y=ltrain)

print("Kernel linear --- ")
print("Best params: %s with score %0.5f" % (grid_lin.best_params_, grid_lin.best_score_))

Kernel linear --- 
Best params: {'C': 0.10000000000000001} with score 0.87245


In [100]:
grid_rbf = GridSearchCV(svm.SVC(kernel='rbf', cache_size=1000), param_grid=param_rbf, cv=crossval)
grid_rbf.fit(X=ftrain, y=ltrain)

print("Kernel RBF --- ")
print("Best params: %s with score %0.5f" % (grid_rbf.best_params_, grid_rbf.best_score_))

Kernel RBF --- 
Best params: {'C': 1.0, 'gamma': 0.01} with score 0.88776
